# Finding the nearest zip code

**Abstract**: Given a list of 'start points', find the nearest neighbor out of a second list of 'end points'<br><br>
**Real World Problem Example**: Marketers could use nearest zip code distancing for approximating the nearest store (end points) for a group of prospects/targets/customjers (start points)<br>
**Problem Details**: Input data are pickled dataframes that contain identifcally formatted columns of 'zip code', 'latitude', and 'longitude'. The 'latitude and 'longitude' geo-coordinates are centroids for each zip code, and each zip code has only one unique set of geo-coordinates.<br><br>
**Approach**: Below is a brute force algorithmic approach. Leveraging numpy's vecotorization capabilites whever possible.<br>
**Heuristic**: Identify the intersection of zip codes between the 'start points' and 'end points', because these 'start points' are already nearest to own zip codes.<br>
**Algorithm**: Deploys and leverages numpy's vectorizations capabilities to calculate and find the min end points for each start point.

In [1]:
import numpy as np
import pandas as pd

# importing dataframes of 'start' and 'end' points
start_points = pd.read_pickle('start_zip_points.pkl').dropna()
start_points = start_points[~start_points.zip.duplicated()]

end_points = pd.read_pickle('end_zip_points.pkl').dropna()
end_points = end_points[~end_points.zip.duplicated()]

# 'start' and 'end' dataframes contain three identical columns
pd.concat([start_points.head(), end_points.tail()])

,zip,lat,lng
0,10965,41.0618,-74.0128
1,32539,30.7748,-86.4631
2,10314,40.5992,-74.1657
3,54115,44.4046,-88.0920
4,53546,42.6527,-88.9492
7885,08831,40.3267,-74.4295
7886,48224,42.4107,-82.9413
7889,16933,41.8191,-77.0667
7898,29472,33.0571,-80.3184
7900,49646,44.7279,-85.0587


In [2]:
# separates the intersection of two arrays of zipcodes
# input of an input array of zipcodes and lookup array of zipcodes
# returns a tuple of inverse boolean arrays that denote matched and unmatched input zipcodes
def match_zipcode(input_zipcodes, lookup_zipcodes):
    match_boolean = input_zipcodes.isin(lookup_zipcodes)
    return match_boolean, ~match_boolean

# input of a start point and an array of end points
# returns an array of calculated arc distances
degrees_to_radians = np.pi/180.0
def arc_distance_calculation(start_point, end_points):
    phi1 = (90.0 - start_point[0])*degrees_to_radians
    phi2 = (90.0 - end_points[:, 0])*degrees_to_radians

    theta1 = start_point[1]*degrees_to_radians
    theta2 = end_points[:, 1]*degrees_to_radians
    
    cos = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + np.cos(phi1)*np.cos(phi2))
    cos = np.minimum(1, cos)
    cos = np.maximum(-1, cos)
    return np.arccos(cos)

# input of a start point and an array of end points
# returns the idx and min distance of nearest end point to start point
def min_end_point_distancing(start_point, end_points):
    distances = np.array(arc_distance_calculation(start_point, end_points))
    idx = distances.argmin()
    return np.array([idx, distances[idx]])

# input of an array of start points and array of end points
# returns the idx and min distance of nearest end point for every start point
def loop_min_end_point_distancing(start_points, end_points):
    _start_points = start_points.loc[:, ['lat','lng']].values 
    return [min_end_point_distancing((x[0], x[1]), end_points.loc[:, ['lat','lng']].values) for x in _start_points]
    
# input of a dataframe of start points and dataframe of end points
# returns a dataframe of start points with nearest end points appended as columns
def get_nearest_zipcode(start_points, end_points, sample=0):
#     used for testing and debugging
    if sample > 0:
        start_points = start_points.sample(sample, random_state=sample)
        end_points = end_points.sample(sample, random_state=sample)
    
#     splitting matched and unmatched zip codes of start points and end points
    matched, unmatched = match_zipcode(start_points.zip, end_points.zip)
    
#     creating output fields for matched zip codes
    matched = start_points[matched]
    matched.loc[:,'nearest_zip'] = matched['zip']
    matched.loc[:,'nearest_lat'] = matched['lat']
    matched.loc[:,'nearest_lng'] = matched['lng']
    matched.loc[:,'nearest_distance'] = 0
        
#     algorithmic min distance search for unmatched zip codes
    unmatched = start_points[unmatched]
    _nearest_idx_distance = np.stack(loop_min_end_point_distancing(unmatched, end_points))
    nearest_end_points = end_points.iloc[_nearest_idx_distance[:,0]].rename(columns={'zip':'nearest_zip',
                                                                                     'lat':'nearest_lat',
                                                                                     'lng':'nearest_lng'})
    nearest_end_points.loc[:,'nearest_distance'] = _nearest_idx_distance[:,1] * 3959 # earth's radius in miles
    unmatched = pd.concat([unmatched.reset_index(drop=True), nearest_end_points.reset_index(drop=True)], axis=1)
    
    return pd.concat([matched, unmatched], ignore_index=True)

results = get_nearest_zipcode(start_points, end_points)

# showing the number of iterations for these two sets of start and end points
matched, unmatched = match_zipcode(start_points.zip, end_points.zip)
print('total_start_points, end_points:', (len(unmatched), len(end_points)))
print('unmatched_start_points, end_points:', (sum(unmatched), len(end_points)))
print('algorithm iterations excluding matched zip codes:', sum(unmatched) * len(end_points))

/Users/daddy/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/daddy/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


total_start_points, end_points: (16438, 4349)
unmatched_start_points, end_points: (12220, 4349)
algorithm iterations excluding matched zip codes: 53144780


In [3]:
pd.concat([results.head(), results.tail()], axis=0)

,zip,lat,lng,nearest_zip,nearest_lat,nearest_lng,nearest_distance
0,10965,41.0618,-74.0128,10965,41.0618,-74.0128,0.000000
1,32539,30.7748,-86.4631,32539,30.7748,-86.4631,0.000000
2,10314,40.5992,-74.1657,10314,40.5992,-74.1657,0.000000
3,54115,44.4046,-88.0920,54115,44.4046,-88.0920,0.000000
4,16137,41.2301,-80.2384,16137,41.2301,-80.2384,0.000000
16433,60603,41.8802,-87.6255,60607,41.8750,-87.6516,1.390038
16434,61769,40.8820,-88.3993,61764,40.8823,-88.6282,11.958149
16435,43469,41.4595,-83.3632,49270,41.8697,-83.6833,32.808367
16436,99362,46.0893,-118.3074,54730,45.0200,-91.7234,1282.361769
16437,76691,31.7782,-97.0963,76684,31.7125,-97.1164,4.690835


In [4]:
%%timeit
# Implementation 1: pd.DataFrame.apply to create 2D numpy arrays for number of start_points
get_nearest_zipcode(start_points, end_points)

11.4 s ± 478 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Take-aways:
I think that this algorithms performance could possibly be further improved by replacing the singular pd.DataFrame.apply step with pure numpy vectorization, which would yield a numpy ndarray of shape (x, y, z) spanning three dimensions. TBD.

## Vectorization
Below is a rework of the above algorithm with a slightly different implementation than the above.<br>
Performance slightly, improved by replacing the singular pd.DataFrame.apply step with pure numpy vectorization.<br>-1.) Produce a giant 2D matrix of shape (len(start_points) * len(end_points), 4) where 4 columns are two sets of lat/lng from star_points and end_points
<br>-2.) Calculate the arc distance for each row of two sets of geo-coordinates returns 1 column from 4 
<br>-3.) Reshape into 2D matrix of shape (len(start_points), len(end_points))
<br>-4.) Find the minimum distance for each row (start_point)
<br>
<br>
Two of the functions from above are modified: arc_distance_calculation() and get_nearest_zipcode()<br>

In [5]:
# vectorized arc distance function
# input of 2d array with 4 columns of lat and lng coordinates
# returns 1d array of calculated arc distances
def _arc_distance_calculation(start_end_points):
    degrees_to_radians = np.pi/180.0
    
    phi1 = (90.0 - start_end_points[:, 0])*degrees_to_radians
    phi2 = (90.0 - start_end_points[:, 2])*degrees_to_radians
    
    theta1 = start_end_points[:, 1]*degrees_to_radians
    theta2 = start_end_points[:, 3]*degrees_to_radians
   
    cos = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + np.cos(phi1)*np.cos(phi2))
    cos = np.minimum(1, cos)
    cos = np.maximum(-1, cos)
    return np.arccos(cos)
 
# reworked function to construct a giant 2d cartesian array for numpy vectorization
# follows the below strategy (marked in code):
# -1.) Construct a 2D cartesian matrix of shape (len(start_points) * len(end_points), 4) for geo lat/lng 
# -2.) Calculate the arc distance for each row of two sets of geo-coordinates returns 1 column from 4 
# -3.) Reshape into 2D matrix of shape (len(start_points), len(end_points))
# -4.) Find the minimum distance for each row (start_point)
def _get_nearest_zipcode(start_points, end_points): 
    matched, unmatched = match_zipcode(start_points.zip, end_points.zip)
    matched = start_points[matched]
    matched.loc[:,'nearest_zip'] = matched['zip']
    matched.loc[:,'nearest_lat'] = matched['lat']
    matched.loc[:,'nearest_lng'] = matched['lng']
    matched.loc[:,'nearest_distance'] = 0
   
    unmatched = start_points[unmatched]
    d0, d1 = len(unmatched), len(end_points)
    start_mtx = np.repeat(unmatched.loc[:, ['lat','lng']].values, repeats=d1, axis=0)
    end_mtx = np.tile(end_points.loc[:, ['lat','lng']].values, reps=(d0,1))
    cartesian_mtx = np.hstack([start_mtx, end_mtx]) # -1.)
    _cartesian_arc_distances = _arc_distance_calculation(cartesian_mtx).reshape((d0, d1)) # -2.) and -3.)
    _nearest_arc_distance_idx = _cartesian_arc_distances.argmin(axis=1) # -4.)
    nearest_end_points = end_points.iloc[_nearest_arc_distance_idx.reshape(d0,)]
    nearest_end_points.loc[:,'nearest_distance'] = _cartesian_arc_distances[np.arange(d0), _nearest_arc_distance_idx] * 3959
    nearest_end_points = nearest_end_points.rename(columns={'zip':'nearest_zip',
                                                            'lat':'nearest_lat',
                                                            'lng':'nearest_lng'})
    unmatched = pd.concat([unmatched.reset_index(drop=True), nearest_end_points.reset_index(drop=True)], axis=1)
   
    return pd.concat([matched, unmatched], ignore_index=True)

_results = _get_nearest_zipcode(start_points, end_points)

In [6]:
pd.concat([_results.head(), _results.tail()], axis=0)

,zip,lat,lng,nearest_zip,nearest_lat,nearest_lng,nearest_distance
0,10965,41.0618,-74.0128,10965,41.0618,-74.0128,0.000000
1,32539,30.7748,-86.4631,32539,30.7748,-86.4631,0.000000
2,10314,40.5992,-74.1657,10314,40.5992,-74.1657,0.000000
3,54115,44.4046,-88.0920,54115,44.4046,-88.0920,0.000000
4,16137,41.2301,-80.2384,16137,41.2301,-80.2384,0.000000
16433,60603,41.8802,-87.6255,60607,41.8750,-87.6516,1.390038
16434,61769,40.8820,-88.3993,61764,40.8823,-88.6282,11.958149
16435,43469,41.4595,-83.3632,49270,41.8697,-83.6833,32.808367
16436,99362,46.0893,-118.3074,54730,45.0200,-91.7234,1282.361769
16437,76691,31.7782,-97.0963,76684,31.7125,-97.1164,4.690835


## Run-time results

In [7]:
%%timeit
# Implementation 1: pd.DataFrame.apply to create 2D numpy arrays for number of start_points
get_nearest_zipcode(start_points, end_points)

11.1 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
# Implementation 2: complete np.array vectorization achieved by constructing a cartesian matrix
_get_nearest_zipcode(start_points, end_points)

7.85 s ± 261 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
print('implementation 2 was ~{0:4.1f}% quicker'.format(abs(7.85/11.1-1)*100))

implementation 2 was ~29.3% quicker
